In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['is', 'for', 'memory', 'best', 'medicine', 'Repeat', 'the']


In [4]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0

In [5]:
print(word2index)

{'is': 1, 'for': 2, 'memory': 3, 'best': 4, 'medicine': 5, 'Repeat': 6, 'the': 7, '<unk>': 0}


In [6]:
print(word2index['memory'])

3


In [7]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'is', 2: 'for', 3: 'memory', 4: 'best', 5: 'medicine', 6: 'Repeat', 7: 'the', 0: '<unk>'}


In [8]:
print(index2word[2])

for


In [9]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

In [50]:
X, Y = build_data(sentence, word2index)

In [51]:
print(X)
print(Y)

tensor([[6, 1, 7, 4, 5, 2]])
tensor([[1, 7, 4, 5, 2, 3]])


In [58]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # print('hidden'+hidden)
        # print('output'+output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [59]:
word2index

{'is': 1,
 'for': 2,
 'memory': 3,
 'best': 4,
 'medicine': 5,
 'Repeat': 6,
 'the': 7,
 '<unk>': 0}

In [60]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5 # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [61]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [62]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)
print(len(output))

tensor([[-1.6828e-01,  4.6725e-01,  2.8142e-01,  7.1506e-04, -4.0798e-02,
          2.1945e-02,  2.5927e-04, -6.0738e-02],
        [ 6.1106e-02,  1.1492e-01, -1.0652e-01,  2.7188e-02, -3.3683e-01,
          4.5132e-01, -4.1672e-02,  1.4692e-01],
        [-1.8238e-01,  3.3871e-01,  3.0813e-01,  1.4546e-01,  2.8443e-02,
          1.4192e-01, -2.2537e-01,  1.4623e-01],
        [-4.1113e-02, -2.0559e-03, -1.1948e-01,  4.3147e-01,  6.5399e-02,
         -8.6609e-03, -3.4161e-01,  1.9259e-01],
        [-8.8748e-02, -1.8765e-01, -3.5921e-01,  8.6758e-02, -1.8308e-01,
          1.4794e-01, -1.6823e-01,  1.5985e-01],
        [-1.7529e-01, -1.2414e-03, -1.8029e-02,  2.9014e-01,  1.9893e-01,
         -2.0259e-01, -5.5253e-01, -9.8083e-02]], grad_fn=<ViewBackward0>)
6


In [63]:
print(output.shape)

torch.Size([6, 8])


In [64]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [65]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print(f"[{step+1}/201] {loss} ")
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[1/201] 2.0178134441375732 
Repeat is medicine is memory the memory

[41/201] 1.413915753364563 
Repeat is the best medicine for memory

[81/201] 0.7892491817474365 
Repeat is the best medicine for memory

[121/201] 0.3985527455806732 
Repeat is the best medicine for memory

[161/201] 0.21290820837020874 
Repeat is the best medicine for memory

[201/201] 0.12889675796031952 
Repeat is the best medicine for memory



In [49]:
X

tensor([[6, 1, 7, 4, 5, 2]])

In [21]:
decode(pred)

['is', 'the', 'best', 'medicine', 'for', 'memory']

In [35]:
X1 = torch.tensor([[1,2,3,4,5,6]])
X1

tensor([[1, 2, 3, 4, 5, 6]])

In [38]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X1)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print(f"[{step+1}/201] {loss} ")
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[1/201] 0.013221792876720428 
Repeat is the best medicine for memory

[41/201] 0.011521746404469013 
Repeat is the best medicine for memory

[81/201] 0.01020114403218031 
Repeat is the best medicine for memory

[121/201] 0.009140086360275745 
Repeat is the best medicine for memory

[161/201] 0.008265900425612926 
Repeat is the best medicine for memory

[201/201] 0.007531801704317331 
Repeat is the best medicine for memory

